# MS141 Lecture 6

# Numerical Integration

Reading material: first part of chapter 5 of Newman's book.<br>
Parts of this notebook are based on the [NumFys](https://www.numfys.net/modules/) notebooks by Gjennestad et al.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Set common figure parameters
newparams = {'figure.figsize': (10, 5), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

Our goal is to numerically compute a definite integral, which can be formally defined as the Riemann sum:

$$\int_a^b\! f(x)\, dx = \lim_{N \rightarrow \infty} \frac{(b-a)}{N} \sum_{i=1}^N\, f(x_i) .$$

In this definition, the integration domain $(b-a)$ is divided into $N \rightarrow \infty$ intervals of equal (and vanishingly small) length, $h = (b-a)/N$, and the function is evaluated at the midpoints $x_i$ of each interval. The contribution to the total area from an interval with midpoint $x_i$ is then $f(x_i)\, h$, and the total value of the integral is the sum of the contributions from the $N \rightarrow \infty$ intervals.

Similar to what we saw with derivatives, when computing an integral numerically, we cannot compute the limit $N\rightarrow \infty$ in the Riemann sum definition, but rather, we need to approximate it as a sum over a finite number of intervals (of finite size). The numerical approach is essential when integrating data rather than an analytic function, and also when a closed form solution to the integral doesn't exist or is hard to calculate. 

In all the approximations discussed below, we will write the numerical integral as:

$$
\boxed{\,\,\, \int_a^b\! f(x)\, dx \,=\, \sum_{i=1}^N\, w_i\,\, f(x_i) \,\,\,}
$$

where the function is evaluated at a finite number $N$ of points $x_i$ (also called "nodes"), and $w_i$ are the weights of points $x_i$. This is a useful formula to remember.<br> 
In practice, different approximations use different choices of the weights and locations of the points $x_i$.

## 1. Midpoint, Trapezoid, and Simpson's methods

There are different ways of approximating the Riemann sum for a finite number $N$ of intervals. The strategy is to find an approximation that is easy to implement and that uses the smallest possible number of points $N$. We want to minimize $N$ for a given accuracy because computing the function at each point and adding these contributions together is computationally demanding and can lead to rounding errors. 

To keep $N$ small and obtain accurate results, we need a method in which the truncation error decreases rapidly with $N$. Three popular approaches based on the Riemann sum definition are the midpoint, trapezoid, and Simpson *quadrature* methods. They are simple to implement. 

### 1.1 Midpoint rule

The midpoint (aka, rectangle) rule is perhaps the simplest and most intuitive numerical integration scheme. It approximates the integral as a sum of rectangles built using the values of the function at the center of the intervals. The goal is to integrate a function on a domain of length $(b-a)$, which is divided into $N$ intervals of equal length, 
$h = (b-a)/N$. The function is then evaluated at the points $x_i$ in the middle of each interval. 

The contribution to the total area from an interval with midpoint $x_i$ is the rectangle area $f(x_i)\, h$, and the total value of the integral is

$$
\boxed{ \int_a^b\!dx\,\,f(x) = \sum_{i=1}^{N} h\,\, f(x_i) }
$$

Each point $x_i$ contributes with an equal weight $w_i = h = \frac{b-a}{N}$. 
The code below visualizes the rectangle method.<br>

In [ ]:
from matplotlib.path import Path
import matplotlib.patches as patches

def draw_trapezoid(xpoints, ypoints):
    """ Function for drawing trapezoids. Takes arrays of x- and y-values as input,
    moving clockwise from bottom left corner. """
    verts = [
        (xpoints[0], ypoints[0]), # left, bottom
        (xpoints[1], ypoints[1]), # left, top
        (xpoints[2], ypoints[2]), # right, top
        (xpoints[3], ypoints[3]), # right, bottom
        (0., 0.), # ignored
        ]

    codes = [Path.MOVETO,
             Path.LINETO,
             Path.LINETO,
             Path.LINETO,
             Path.CLOSEPOLY,
             ]

    path = Path(verts, codes)
    ax = plt.gca()
    patch = patches.PathPatch(path, facecolor='#d3d3d3', lw=1)
    ax.add_patch(patch)

In [ ]:
x = np.linspace(0,1,100)
def f(x): return x**2

plt.plot(x, f(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')

# Plot rectangles:
N = 12
h = 1/(N-1)
for n in range(1,N):
    draw_trapezoid([(n-1)*h,(n-1)*h,n*h,n*h], [0,f((n-0.5)*h),f((n-0.5)*h),0])

Let's apply the midpoint rule to compute the integral of a Gaussian function:

$$ I(z) = \frac{2}{\sqrt{\pi}} \int_0^z\!dx\,\, e^{-x^2} \,=\, \mathrm{erf}(z)$$

This well-known integral is the error function, $\mathrm{erf}(z)$, computed below at $z=1$. Its exact value can be accessed using `scipy.special.erf`, so we can check the result of our numerical integration. Note that this integral cannot be computed analytically, so this is a good test case for our numerical approaches.

In [ ]:
# Midpoint rule implementation

from scipy.special import erf

# number of steps, interval, increment
N = 100
a = 0 
b = 1.0 
h = (b - a)/N # h=0.01
x = np.linspace(a,b,N+1) # N=100 intervals

# Midpoints
# x_mid = (x_right + x_left)/2
x_mid = (x[1:] + x[:N]) / 2.0
#debug
#print(x_mid)

# Integrand, 2/sqrt(pi) e^-x^2, evaluated at the midpoints
y_mid = (2.0 / np.sqrt(np.pi) )* np.exp(-x_mid**2)

# Integral
mid_int = np.sum(y_mid)*h
print('result = ', mid_int)

# exact result, error (test as a function of N)
exact = erf(1.0)
mid_error = abs(mid_int - exact)
print ('error  = ', mid_error)

### 1.2 Trapezoid Rule

From the plot above, it's clear that the midpoint method makes a fairly large error in the integral. 
One way to improve it is to increase the number of intervals and make the intervals smaller, since in the limit of large $N$ we should approach the exact value of the integral. However, increasing the number of intervals makes the calculation more expensive, while round-off errors will ultimately become dominant in the large $N$ limit.

A better approximation is to use trapezoids instead of rectangles, so we approximate the function as linear in each interval. The two sides (left and right) of each trapezoid are determined by the interval length $h$, while the other two sides (top and bottom) are determined by the values of the function $f(x)$ at the two ends of the interval. The method is visualized below.

In [ ]:
x = np.linspace(0,1,100)
def f(x): return x**2

plt.plot(x, f(x))
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')

# Plot trapezoids:
N = 4 # try N=4
h = 1/(N-1)
for n in range(1,N):
    draw_trapezoid([(n-1)*h,(n-1)*h,n*h,n*h], [0,f((n-1)*h),f((n)*h),0])


We can see that the trapezoid method amounts to **linearizing the function in the interval**, so that any residual error in the integral comes from the curvature of the function (try setting $N = 4$ in the plot above to see this trend more clearly). It seems that overall we are making a smaller error than before; however, note that the midpoint method takes advantage of error cancellation.

Let us derive a formula for the area covered by the trapezoids. The area of each trapezoid is 

$$A_n \,=\, \frac{f_n+f_{n+1}}{2} \,h $$

where the grid points are $x_n = n \,h$, and $f_n = f(x_n)$; the base of the trapezoid is $h = \frac{b-a}{N}$ 
and $\frac{f_n+f_{n+1}}{2}$ its average height.

We use $N+1$ points, of which the first and last are the interval endpoints, so that we have $N$ intervals and trapezoids.<br> Adding up all the trapezoids, the total area is

$$A=A_1 \,+\, A_2 \,+\, A_3 \,+\,...\,+\,A_{N}$$

and thus

$$A=h\, \left( \frac{f_1 +f_2}{2} + \frac{f_2 +f_3}{2} + \frac{f_3 +f_4}{2} + \ldots \frac{f_{N} +f_{N+1}}{2} \right) $$

As a result, we can approximate the integral as

$$\boxed{ \int_a^b\!dx \,\, f(x) \approx h\, \left(\frac{1}{2}\,f_1 + f_2 + f_3 + \ldots + f_N + \frac{1}{2}\,f_{N+1}\right) }$$

We can see that the trapezoid method corresponds to using weights $w_1 = w_{N+1}= h/2$ for the endpoints, and $w_i = h$ for the other points in the grid.<br> 
The nodes are the grid points $x_n$ rather than the midpoints. The sum of the weights is the number of intervals times $h$, as it should. <br> 

The implementation of the trapezoid rule is simple if we start from the midpoint rule code. We apply the trapezoid method to the calculation of the error function discussed above, and find that the error is similar to the midpoint rule.

In [ ]:
# Trapezoid method implementation

from scipy.special import erf

# number of steps, intervals, increment
N = 100
a = 0 
b = 1 
h = (b - a)/N
x = np.linspace(a,b,N+1)

# integrand
y = (2.0 / np.sqrt(np.pi) )* np.exp(-x**2)

# Integral  
# contributions from the endpoints
trap_int = (y[0] + y[N])*h / 2.0
# and from the other points
trap_int += np.sum(y[1:N])*h

print ('result = ', trap_int)

# exact result, error (test as a function of N)
exact = erf(1.0)
trap_error = abs(trap_int - exact)
print ('error  = ', trap_error)

### 1.3 Simpson's rule

There are more accurate methods than the midpoint and trapezoid rules to approximate an integral. Intuitively, to improve the result we need to take into account the curvature of the function in the subintervals. 

Simpson's rule relies on this intuition and approximates the integrand function with a quadratic polynomial locally in each interval: 

$$f(x) \approx \alpha x^2 + \beta x + \gamma $$

The parameters $\alpha, \beta, \gamma$ are chosen in a clever way. Consider **two adjacent intervals**, one between $x_i - h$ and $x_i$ and the other between $x_i$ and $x_i + h$. We can write: 

$$ \int_{x_i - h}^{x_i + h}\!dx \,\, f(x) = \int_{x_i - h}^{x_i}\!dx \,\, f(x) + \int_{x_i}^{x_i + h}\!dx \,\, f(x)$$

By substituting $\,f(x) = \alpha x^2 + \beta x + \gamma $ and integrating, we express the result of the integral in terms of the function at the two endpoints $x_i \pm h$ of the double interval and at its center at $x_i$ [below, $f_i = f(x_i)$]:

$$ \int_{x_i - h}^{x_i + h}\!\!dx \,\, f(x) \,=\, \frac{h}{3}\, f_{i-1} + \frac{4h}{3}\, f_i + \frac{h}{3}\, f_{i+1} $$

It is important to note that Simpson's rule requires the elementary integration to be over pairs of intervals, which in turn requires an even number of intervals, or equivalently, an *odd number of points* ($\,N = 3, 5, 7, \ldots$) in the integration grid. 

To apply Simpson's rule to the entire integration domain, we add up the contributions from all interval pairs, counting twice all contributions from the interval endpoints (but only once the domain endpoints): 

$$ \boxed{ \int_a^b \!\! dx \,\,f(x) \,=\, \frac{h}{3}\, f_{1} \,+\, \frac{4h}{3}\, f_2 \,+\, \frac{2h}{3}\, f_3  
\,+\, \frac{4h}{3}\, f_4 \,+\, \ldots \,+\, \frac{4h}{3}\, f_{N-1} \,+\, \frac{h}{3}\, f_{N} }$$

We find that the weights are $w_1 = w_N = \frac{h}{3}$ for the domain endpoints, $w_k = \frac{4h}{3}$ for $k$ even, and $w_k = \frac{2h}{3}$ for $k$ odd (see the figure below).<br> 
An important check is that the weights add up to $h$ times the number of intervals:

$$ \sum_{k=1}^N w_k = h\, (N - 1) $$

A simple example to illustrate Simpson's rule uses five points: two "double intervals" with one midpoint each. 
In this case, we obtain

$$\int_a^b f(x) \mathrm{d}x \approx\, \frac{h}{3}\,\left[ \, f(x_1)+4f(x_2)+2f(x_3)+4f(x_4)+f(x_5) \,\right],$$

where $h$ is the interval length of a regular $x$-axis grid, as usual. This scenario is plotted below.

In [ ]:
a = 0.0
b = 1.0
N = 5
h = (b-a)/(N-1)
x = np.linspace(a,b,N)

# pick a function for visualization purposes
y = x**3 + 0.5

# Plot lines
color = ['b', 'g', 'r', 'g', 'b']
for i in range(N):
    plt.plot([i*h, i*h],[0, y[i]], color[i], lw=2)

plt.plot(x, y,lw=2, c='orange')
plt.ylabel(r'$f(x)$')
plt.xlabel(r'$x$')

# show weights
plt.text(0.015, 0.1, r'$\frac{h}{3}$', color='blue',fontsize=22)
plt.text(0.265, 0.1, r'$\frac{4h}{3}$', color='g',fontsize=22)
plt.text(0.505, 0.1, r'$\frac{2h}{3}$', color='r',fontsize=22)
plt.text(0.755, 0.1, r'$\frac{4h}{3}$', color='g',fontsize=22)
plt.text(0.965, 0.1, r'$\frac{h}{3}$', color='b',fontsize=22)

plt.show();

Let us now implement Simpson's rule for an arbitrary value of $N$ and apply it to the $erf$ calculation discussed above.

In [ ]:
# Simpson rule implementation

from scipy.special import erf

a = 0.0           
b = 1.0           
N = 101 # an odd number of points           
h = (b - a)/(N-1) # an even number of intervals of length h
x = np.linspace(a,b,N)

# integrand
y = (2.0 / np.sqrt(np.pi) )* np.exp(-x**2)

#debug
#print (y,'\n') # all the elements
#print (y[1:N:2],'\n') # even elements only, no endpoints
#print (y[2:N-1:2]) # odd elements only, no endpoints

# weights
wend = h / 3.0
weven = 4.0 * h / 3.0
wodd = 2.0 * h / 3.0

# Integral - contributions from:
# endpoints
sim_int = (y[0] + y[N-1]) * wend

# even points (no endpoints)
sim_int += np.sum(y[1:N:2]) * weven

# odd points (no endpoints)
sim_int += np.sum(y[2:N-1:2]) * wodd


print ('result = ', sim_int)

# exact result, error (test as a function of N)
exact = erf(1.0)
sim_error = abs(sim_int - exact)
print ('error  = ',sim_error)

We obtain a higher accuracy than in the rectangle and trapezoid rule. Simpson's method is a simple integration approach that achieves a higher accuracy. It accounts for local curvature by properly distributing the integration weights on the grid.

**Higher-order methods.** More advanced integration methods than those discussed above include Romberg integration, Gaussian quadrature, and adaptive quadrature approaches.  These methods achieves a smaller error at the cost of a more complex implementation. In Gaussian quadrature, both the nodes (points $x_n$) and the weights are optimally chosen to minimize the error, resulting in tables of nodes and weights derived using orthogonal polynomials. Romberg integration uses Richardon's extrapolation to minimize the error. Both approaches are suitable mainly for smooth functions. Adaptive integration adjusts the subinterval in different regions of the integration domain, achieving a higher accuracy at the cost of a more complex implementation. Newman's book has a good discussion of these higher-order integration methods, which are typically useful only in particular cases and mainly in one-dimensional integrals.

### 1.4 Error analysis

Our results for $N \approx 100$ integration points suggest that the errors of the midpoint and trapezoid rules are comparable, while the error of the Simpson rule is significantly smaller:

In [ ]:
print ('Error midpoint =', mid_error)
print ('Error trapezoid =', trap_error)
print ('Error Simpson   =', sim_error)

Let us estimate the error of the midpoint rule by comparing the exact integral $\int f(x)dx$ with its midpoint rule approximation. We write the midpoint rule error $E_{\rm{mid}}$ as $N$ times the local error in each interval:

$$ E_{\rm{mid}} \approx N e_{\rm{loc}} $$

with 

$$ e_{\rm{loc}} = \int_{x_i}^{x_i + h}\!\!dx\,\, f(x) \,-\, h\, f(x_m) $$

where $x_m = x_i + \frac{h}{2}$ is the midpoint of the interval.
We expand the integrand $f(x)$ in Taylor series about the midpoint, and integrate over the interval:

$$ e_{\rm{loc}} = \int_{x_i}^{x_i + h}\!\!dx\,\, \left[\, f(x_m) + f'(x_m) (x-x_m) + \frac{f''(x_m)}{2} (x-x_m)^2  + \ldots \,\right] \,-\, h\, f(x_m). $$

The term $f(x_m)$ in the integrand cancels the midpoint result, while the second term in brackets gives zero once integrated.<br> Integrating the third term in brackets gives $|f''(x_m)| h^3 / 24$, where $h = (b-a)/N$. Therefore, we can approximate the midpoint error as:

$$ E_{\rm{mid}} = N e_{\rm{loc}} = |\,f''(x_{\rm max})\,| \frac{(b-a)^3}{24 N^2}$$

Using a similar approach, we can obtain the errors of all three quadrature methods we examined so far as:
1. Midpoint, $E = \frac{(b-a)^3}{24\,N^2}|f^{(2)}(\eta)|$


2. Trapezoid, $E = \frac{(b-a)^3}{12\,N^2} |f^{(2)}(\eta)|$


3. Simpson, $E = \frac{(b-a)^5}{180\,N^4} |f^{(4)}(\eta)|$

where $\eta$ is the point at which the second derivative $f^{(2)}(x)$ (or the fourth derivative for the Simpson method) is maximal.

We see that the errors in the midpoint and trapezoid cases are similar, and are both proportional to the curvature times $N^{-2}$. The midpoint error is half of the trapezoid error in spite of its simple approximation. The error in the Simpson method is smaller and it decreases much faster with $N$ than in the other two methods. This implies that the Simpson rule has a superior accuracy for a given value of $N$, which is consistent with our numerical results.

Note also that while the midpoint and trapezoid results can be improved by increasing the number of grid points $N$, due to round-off errors (see the Lecture on numerical error) each method has an ideal number of points $N$, beyond which the error becomes dominated by round-off errors. Choosing a very large value of $N$ is thus not a good idea, both because it increases computation time and because it can lead to larger round-off errors. The key is to choose an accurate method, not to over-sample.

## 2. Monte Carlo integration

A completely different approach to integration, inspired by random numbers and the laws or probability, is the so-called Monte Carlo approach.<br> 
The standard Monte Carlo technique relies on the mean value theorem, which in one dimension reads:

$$ I = \int_a^b \!\! dx\,\, f(x) = (b-a)\, \langle\, f \,\rangle $$

where $\langle f\rangle$ is the average value of the function over the integration domain $(b-a)$. The theorem states the obvious if you think of integrals as areas $-$ the area under the curve $f(x)$ can be computed as the length of the interval $(b-a)$ times a number, which is the average height $\langle f\rangle$ of the curve $f(x)$.

The Monte Carlo integration algorithm uses *random points* to evaluate the mean value $\langle f\rangle$. Using a sequence of $N$ points $x_i$ chosen randomly in the integration domain, so that $a \le x_i \le b$, we determine the mean value of $f$ by sampling the function at these points:

$$ \langle\, f \,\rangle \approx \frac{1}{N} \sum_{i=1}^N\, f(x_i) \,,$$

which gives us the Monte Carlo integration rule in one dimension:

$$ \boxed{ \int_a^b \!\! dx\, f(x) \approx \, I_N \, = (b-a) \frac{1}{N} \sum_{i=1}^N\, f(x_i) }.$$

This is similar to our quadrature integration rule, but with points $x_i$ chosen randomly, and equal weights $w_i = h = \frac{b-a}{N}$. 

Why do we expect this approach to work at all? The reason is the [law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers) (LLN). If we let the number of samples of $f(x)$ approach infinity, so that $N \rightarrow \infty$, or if we take the average of infinitely many runs for a finite $N$, the LLN guarantees that the sum $\frac{1}{N} \sum_{i=1}^N\, f(x_i)$ will approach the mean of $f(x)$.

If we interpret $f(x)$ as a probability distribution, the values $f_i = f(x_i)$ at the random points $x_i$ are independent and identically distributed random variables.<br> 
The LLN states that if we consider a sequence $f_1$, $f_2$, $\ldots$ of independent identically distributed random variables with mean $\langle f \rangle$ and variance $\sigma^2_f$, and form the **sample mean**

$$ M_N = \frac{f_1 + f_2 + \ldots + f_N}{N} ,$$

then the mean value of $M_N$ is $\langle f \rangle$ (which is the result we want) and its variance is $\sigma^2 = \sigma^2_f\,/\, N $. Therefore, in the limit of large $N$, this result guarantees that the standard deviation of the sample mean tends to zero as $1/\sqrt{N}$: 

$$ \sigma = \frac{\sigma_f}{\sqrt{N}} $$

Since the result of the Monte Carlo integration is $I_N = (b-a)\, M_N$, the theorem guarantees that in the limit of large $N$, the numerically computed integral $I_N$ will approach the exact integral $I$: 

$$ \lim_{N \rightarrow \infty}\,\, I_N = \lim_{N \rightarrow \infty}\,\,(b-a) \frac{1}{N} \sum_{i=1}^N\, f(x_i) = \int_a^b \!\!dx\,\, f(x) $$

The decrease in the standard deviation as $N^{-1/2}$ guarantees that the computed integral $I_N$ on the lefthand side above will approach the exact result $I$ (the righthand side above) with a convergence rate proportional to $N^{-1/2}$ (provided round-off errors don't become dominant).

This may seem slow compared to the faster $N^{-2}$ convergence of the midpoint or trapezoid rules, but in *higher dimensions* (see below), Monte Carlo becomes the fastest converging scheme and the most practical and easiest to implement.

Let us implement Monte Carlo integration in 1 dimension, and apply it to compute a few integrals.

In [ ]:
# Monte Carlo integration in 1D

def montecarlo(f, a, b, N):
    h = (b - a) / N
    # array of N random points in (a,b)
    # generate N uniformly distributed random points between (a,b)
    R = np.random.random(N)*(b - a) + a   # print out if you want
    return np.sum(f(R))*h

Yes $-$ it's as simple as that! Let's apply it to the $erf(1)$ calculation discussed above.

In [ ]:
def f(x):
    return (2.0/np.sqrt(np.pi))*np.exp(-x**2)

#
mc_int = montecarlo(f,0.0,1.0,100)
print ('result = ', mc_int)

exact = erf(1.0)
mc_error = abs(mc_int - exact)
print ('error  = ', mc_error)

We find that for $N = 100$ points, the Monte Carlo error is roughly 1000 times larger than for the midpoint method.<br> 
If we want to decrease the error by 1000x, we need to increase the sampling by a factor of $10^6$!<br> 
Let's study explicitly the convergence to the exact result, which is expected to have a $N^{-1/2}$ trend.

In [ ]:
def f(x):
    return (2.0/np.sqrt(np.pi))*np.exp(-x**2)

# an array of integers to test the dependence of the Monte Carlo result
tests = np.array([1.e1, 30, 1.e2, 3.e2, 1.e3, 1.e4, 1.e5, 1.e6, 1.e7], dtype=int)

# an array of floats of the same size to put the integration result and its error
mc_results = np.empty(len(tests), dtype=float)
mc_stdev = np.empty_like(mc_results)

for n in range(len(tests)):
    mc_results[n] = montecarlo(f,0.0,1.0,tests[n])
    mc_stdev[n] = 1.0 / ((np.sqrt(2)) * np.sqrt(tests[n])) # note that stdev sigma_f = 1/sqrt(2)
    #debug
    #print (mc_results[n], mc_stdev[n])

plt.errorbar(tests,mc_results, yerr=mc_stdev, marker='o',markersize=8.0, capsize=0.0, elinewidth=2.0, ecolor='r');
plt.xscale('log')
plt.ylim(0.7,1.0) # try changing this if you want
plt.hlines(erf(1.0), 0.0,1.e7, colors='k', linestyles='dashed') # show exact result with a dashed line
plt.show();

Note that each realization of the Monte Carlo code above will give different results (why?). The $N^{-1/2}$ convergence is realized only on average, namely over an infinite number of runs. The strength of the method lies in the fact that as $N$ becomes large, it becomes less and less likely to have an error greater than $\sigma_f/\sqrt{N}$. This guarantees that the result can be improved by increasing the number of samples.

We covered the very basics of the Monte Carlo method, but there are some important points we will not discuss in detail to keep in mind. 

Since Monte Carlo integration relies on random numbers, for **functions that vary rapidly in the integration domain, or that are large only in a small portion of the integration domain, Monte Carlo may perform poorly** since many random points may end up sampling irrelevant regions in which the integrand function is very small, and may miss the regions where it's large, thus failing to approximate the average value of $f(x)$. To address this problem, **importance sampling** techniques have been devised, as well as methods to generate random points for these importance sampling techniques. Chapter 5 of Landau's book discusses these topics in sufficient detail.

## 3.  Integration in multiple dimensions

Accurate quadrature rules such as Simpson or Gaussian are typically difficult to derive and implement in more than one dimension and not routinely used to compute multi-dimensional integrals. For a rectangular grid in 2D or 3D, it is common to just use the midpoint rule or Monte Carlo integration.

We now ask how the error in the different methods we studied scales when integrating in higher dimensions.<br>
We want to compute an integral in $d$ dimensions,

\begin{equation}
I_d = \int dx_1 \int dx_2 \cdots \int dx_d\,\, f(\mathbf{x}).
\end{equation}

With a quadrature method, we break up the $d$-dimensional space into a grid with $N$ points in each dimension, 
for a total of $N^d = M$ points in $d$ dimensions at which we evaulate our function. 
If the error in our chosen quadrature method scales as $N^{-k}$ in 1 dimension (we refer to it as an order $k$ scheme), then using this method in $d$ dimensions will lead to an error:

\begin{equation}
\mathrm{Error} \sim \frac{1}{N^k} = \frac{1}{M^{k/d}},
\end{equation}

where we used $N = M^{1/d}$. 
This is bad news, since an order $k$ scheme in 1-dimension becomes an order $k/d$ scheme in $d$-dimensions.

By contrast, the error in the Monte Carlo method scales still as $M^{-1/2}$ in $d$ dimensions. It is clear therefore that in higher dimensions Monte Carlo integration becomes preferrable. For example, since the rectangle rule has $k = 2$, it has an error $\sim M^{-2/3}$ in 3 dimensions and $\sim M^{-1/2}$ in 4 dimensions, so that formally for $d > 3$ Monte Carlo integration becomes preferrable. In practice, Monte Carlo can often be preferrable also in 3D (and sometimes even in 2D).

There are more subtle reasons to consider in addition to pure error scaling. Monte Carlo integration is straigthforward to implement, and using importance sampling, one can dramatically speed up its convergence. In addition, one can check the convergence of the integral by averaging over multiple calculations, rather than running calculations with large regular grids using quadrature methods. 

Another thing to keep in mind is that grids in 2D and 3D may employ polyhedra of different kinds with regular or irregular connectivity, leading to complex structured or unstructured grids. For finite element methods, there is a whole subfield of [grid generation](https://en.wikipedia.org/wiki/Principles_of_grid_generation) techniques, and in general the integration method chosen in 3D can be domain specific and is the subjects of books and specialized articles. A good reference is:<br> 
A. Stroud, "Approximate calculation of multiple integrals", Prentice-Hall 1971.

As a last point, we take up the challenge on page 109 of Landau's book to compute the 10-dimensional integral

$$ I = \int_0^1\!dx_1 \int_0^1\!dx_2 \ldots \int_0^1\!dx_{10} \,\,\left(\,x_1 + x_2 + \ldots + x_{10} \,\right)^2 $$

whose exact value is known to be 155/6. We can compute it with just a few lines of code.

In [ ]:
# our target function
# x has to be a 10-dimensional numpy array
def f(x):
    return (np.sum(x))**2

# adapt our monte carlo routine to 10 dimensions
def montecarlo10(f, N): # b=1, a=0
    h = 1.0 / N  # hypervolume of one point: 1^10/N = 1/N
    # array of N random points in (a,b)
    my_sum = 0.0
    # use a scalar f for simplicity
    for i in range(N):
        R = np.random.random(10)
        my_sum += f(R)*h 
    return my_sum

In [ ]:
# run the calculation

mc_int10d = montecarlo10(f,10000)
print ('Monte  Carlo =', mc_int10d)
print ('Exact result =', 155./6)

The result is accurate and the implementation is simple! In contrast, using the midpoint rule would require setting up a 10 dimensional grid, and computing the function at each grid point. Even if you used 10 points per dimension, this would require $N = 10^{10}$ calculations, and the result would converge with a rate of $N^{-2/10} = N^{-1/5}$. Using Monte Carlo, we converged the integral with $N=10000$ points with a relatively small error.